# Topic Modeling

Topic models have been designed specifically for the purpose of extracting various distinguishing concepts or topics from a large corpus having various types of documents where each document talks about one or more concepts. These concepts can be anything from thoughts, opinions, facts, outlooks, statements and so on. The main aim of topic modeling is to use mathematical and statistical techniques to discover hidden and latent semantic structures in a corpus. 
Topic modeling involves extracting features from document terms and using mathematical structures and frameworks like matrix factorization and SVD to generate clusters or groups of terms which are distinguishable from each other and these cluster of words form topics or concepts. These concepts can be used to interpret the main themes of a corpus and also make semantic connections amongst words which co-occur together frequently in various documents. There are various frameworks and algorithms to build topic models. The most popular ones include
- Latent Semantic Indexing 
- Latent Dirichlet Allocation 
- Non-negative Matrix Factorization

The last technique we will look at is non-negative matrix factorization (NNMF), which is another matrix decomposition technique similar to SVD but operates on non-negative matrices and works well for multivariate data.  NNMF can be formally defined as, given a non-negative matrix V, the objective is to find two non-negative matrix factors, W and H such that when they are multiplied, they can approximately reconstruct V. Mathematically this is represented by 
$$ V ≈ WH $$

such that all three matrices are non-negative. 

To get to this approximation, we usually use a cost function like the Euclidean distance or L2 norm between two matrices or the Frobenius norm which is a slight modification of the L2 norm. 

This implementation is available in the NMF class in the scikit-learn decomposition module which we will be using in the section.

# Import necessary dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import nltk

warnings.filterwarnings("ignore")

In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 3.8MB/s 
     |████████████████████████████████| 245kB 8.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=794b9838e69308a2524103dd2115d149d6209ffb97573f7fb9874f064e160f72
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
import nltk
import numpy as np

ps = nltk.porter.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('but')
stop_words.remove('not')


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


def pre_process_document(document):
    
    # strip HTML
    document = strip_html_tags(document)
    
    # lower case
    document = document.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    document = remove_accented_chars(document)
    
    # expand contractions    
    document = expand_contractions(document)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)        
    
    # remove stopwords
    document = remove_stopwords(document, is_lower_case=True, stopwords=stop_words)
        
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load and normalize data

In [ ]:
dataset = pd.read_csv(r'/content/drive/My Drive/nlp/nlp_workshop_dhs18-master/nlp_workshop_dhs18-master/Unit 14 - Topic Modeling/movie_reviews.csv.bz2', compression='bz2')

# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# normalize datasets
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
norm_reviews=np.concatenate((norm_train_reviews, norm_test_reviews), axis=0)

# Extract features from positive and negative reviews

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# consolidate all normalized reviews
# = norm_train_reviews+norm_test_reviews
# get tf-idf features for only positive reviews
positive_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'positive']
ptvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
ptvf_features = ptvf.fit_transform(positive_reviews)
# get tf-idf features for only negative reviews
negative_reviews = [review for review, sentiment in zip(norm_reviews, sentiments) if sentiment == 'negative']
ntvf = TfidfVectorizer(use_idf=True, min_df=0.02, max_df=0.75, ngram_range=(1, 2), sublinear_tf=True)
ntvf_features = ntvf.fit_transform(negative_reviews)
# view feature set dimensions
print(ptvf_features.shape, ntvf_features.shape)

(25000, 893) (25000, 912)


In [ ]:
!pip install pyLDAvis
!pip install django-model-utils

# Topic Modeling on Reviews

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import NMF
#import topic_model_utils as tmu
#import django-model-utils
pyLDAvis.enable_notebook()
total_topics = 10

## Display and visualize topics for positive reviews

In [ ]:
# build topic model on positive sentiment review features
pos_nmf = NMF(n_components=total_topics, solver='cd', max_iter=500,
               random_state=42, alpha=.1, l1_ratio=.85)
pos_nmf.fit(ptvf_features)      
# extract features and component weights
pos_feature_names = np.array(ptvf.get_feature_names())
pos_weights = pos_nmf.components_
# extract and display topics and their components
pos_feature_names = np.array(ptvf.get_feature_names())
feature_idxs = np.argsort(-pos_weights)[:, :15]
topics = [pos_feature_names[idx] for idx in feature_idxs]
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print(', '.join(topic))
    print()

Topic #1:
like, would, think, but, know, people, really, get, say, see, could, no, even, want, something

Topic #2:
movie, movies, great, love, movie not, watch, see, great movie, story, recommend, see movie, loved, watch movie, wonderful, movie but

Topic #3:
show, series, episode, episodes, tv, shows, season, television, great, characters, watch, watching, new, love, every

Topic #4:
good, well, but, really, acting, pretty, plot, action, also, bad, better, job, quite, cast, done

Topic #5:
best, ever, seen, one best, one, ever seen, movies, greatest, made, amazing, never, not seen, performance, films, brilliant

Topic #6:
life, man, story, one, two, but, young, also, character, no, world, new, time, way, scene

Topic #7:
film, films, great, film not, see, great film, film but, wonderful, made, acting, cinema, music, watch, recommend, characters

Topic #8:
saw, years, first, dvd, ago, time, remember, years ago, still, first time, old, saw movie, video, since, watched

Topic #9:
funny,

In [45]:
ptvf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.75, max_features=None,
                min_df=0.02, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
pyLDAvis.sklearn.prepare(pos_nmf, ptvf_features, ptvf, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.074711  0.192247       1        1  28.100667
0      0.048066  0.303929       2        1  18.035315
3      0.199245  0.037193       3        1  11.325083
1     -0.215836 -0.058079       4        1   8.885818
6     -0.036303 -0.198320       5        1   8.763268
7     -0.219861  0.345886       6        1   7.376731
4     -0.219015 -0.394600       7        1   5.358544
2     -0.398019  0.063798       8        1   5.121199
8      0.274047 -0.359130       9        1   4.030968
9      0.492965  0.067076      10        1   3.002408, topic_info=       Term         Freq        Total Category  logprob  loglift
269    film  2630.000000  2630.000000  Default  30.0000  30.0000
488   movie  2408.000000  2408.000000  Default  29.0000  29.0000
298   funny  1491.000000  1491.000000  Default  28.0000  28.0000
69     best  1513.000000  1513.000000  Default  27.0000  27.0000
129  comedy  1086.000000  1086.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
491  movies    52.991177  1131.612842  Topic10  -4.5356   0.4445
273   films    53.129652  1215.285939  Topic10  -4.5330   0.3758
5    acting    37.366600   685.932676  Topic10  -4.8849   0.5958
582    plot    23.204610   443.674945  Topic10  -5.3614   0.5550
516      no    21.990920   438.242863  Topic10  -5.4151   0.5136

[492 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.087089  absolutely
1         2  0.145149  absolutely
1         4  0.236385  absolutely
1         5  0.145149  absolutely
1         6  0.008294  absolutely
...     ...       ...         ...
889      10  0.052977         yet
890       1  1.004279        york
891       1  0.908033       young
891       4  0.007862       young
891       6  0.082548       young

[1383 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 4, 2, 7, 8, 5, 3, 9, 10])

## Display and visualize topics for negative reviews

In [ ]:
# build topic model on negative sentiment review features
neg_nmf = NMF(n_components=total_topics, solver='cd', max_iter=500,
              random_state=42, alpha=.1, l1_ratio=.85)
neg_nmf.fit(ntvf_features)      
# extract features and component weights
neg_feature_names = ntvf.get_feature_names()
neg_weights = neg_nmf.components_
# extract and display topics and their components
neg_feature_names = np.array(ntvf.get_feature_names())
feature_idxs = np.argsort(-neg_weights)[:, :15]
topics = [neg_feature_names[idx] for idx in feature_idxs]
for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print(', '.join(topic))
    print()

Topic #1:
but, one, no, like, would, get, also, characters, story, character, well, two, way, time, first

Topic #2:
movie, bad, movies, watch, acting, good, even, like, but, movie not, really, see, would, watching, could

Topic #3:
film, films, film not, acting, bad, film but, good, script, but, awful, made, plot, watch, director, poor

Topic #4:
ever, worst, seen, ever seen, movie ever, worst movie, one worst, ever made, one, movies, made, worse, possibly, horrible, far

Topic #5:
horror, budget, low, low budget, horror movie, gore, blood, flick, scary, killer, monster, genre, dead, movies, cheap

Topic #6:
book, read, novel, story, based, version, movie, characters, reading, disappointed, made, love, completely, written, original

Topic #7:
show, funny, comedy, not funny, jokes, tv, episode, shows, series, humor, watch, laugh, stupid, television, kids

Topic #8:
waste, waste time, time, not waste, money, complete, watching, please, total, wasted, crap, horrible, nothing, spent, not 

In [ ]:
pyLDAvis.sklearn.prepare(neg_nmf, ntvf_features, ntvf, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.057621  0.051491       1        1  40.824789
1      0.013261 -0.030233       2        1  17.711647
2      0.003701  0.299831       3        1   9.963090
6      0.020994 -0.392536       4        1   5.914034
3     -0.314239  0.319817       5        1   5.645983
9      0.301612  0.383684       6        1   5.548849
4      0.364376 -0.289528       7        1   4.346332
7     -0.470246 -0.019063       8        1   3.931929
8      0.429739  0.020832       9        1   3.111352
5     -0.291576 -0.344294      10        1   3.001995, topic_info=          Term         Freq        Total Category  logprob  loglift
500       much  2463.000000  2463.000000  Default  30.0000  30.0000
264       film  2934.000000  2934.000000  Default  29.0000  29.0000
68      better  2219.000000  2219.000000  Default  28.0000  28.0000
76        book  1482.000000  1482.000000  Default  27.0000  27.0000
854      waste  1555.000000  1555.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
112  character    45.460049   332.503780  Topic10  -4.7183   1.5161
450       made    54.218500   797.102441  Topic10  -4.5421   0.8179
460       many    42.941137   424.994167  Topic10  -4.7753   1.2136
322      great    42.156122   487.551999  Topic10  -4.7937   1.0579
895      would    33.540853   860.551921  Topic10  -5.0223   0.2611

[493 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.248337  absolutely
1         2  0.298846  absolutely
1         3  0.063137  absolutely
1         4  0.012627  absolutely
1         5  0.256755  absolutely
...     ...       ...         ...
908       1  0.850973       years
908       5  0.151284       years
910       1  1.002882         yet
911       1  0.968087       young
911       7  0.037718       young

[1144 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 7, 4, 10, 5, 8, 9, 6])